## Imports and pip install

In [1]:
#install the diffuser package
#pip install --upgrade pip
!pip install --upgrade diffusers transformers scipy
!pip3 install SpeechRecognition pydub
#load the model from stable-diffusion model card
import torch
from diffusers import StableDiffusionPipeline

from huggingface_hub import notebook_login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 51.2 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Get stable diffusion video model

In [2]:
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [3]:
!pip install -U stable_diffusion_videos

from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
#Making Videos
from stable_diffusion_videos import StableDiffusionWalkPipeline
import torch
#"CompVis/stable-diffusion-v1-4" for 1.4

SDpipeline = StableDiffusionWalkPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    revision="fp16",
).to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


## Get text from wav audio file

In [5]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            r.adjust_for_ambient_noise(source,duration=1)
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [8]:
path = "/content/subtitles_trim_audio.wav"
FullTextAudio = get_large_audio_transcription(path)
print("\nFull text:", FullTextAudio)

Error: 
Error: 
audio-chunks/chunk3.wav : And i would see these commercials and i was watching other children suffer in other parts of the world. 
audio-chunks/chunk4.wav : You know the commercials. 
Error: 
audio-chunks/chunk6.wav : 5 cent and save a child's life and you know. 
audio-chunks/chunk7.wav : To myself like i wonder how many 25 cents i could save up to. 
audio-chunks/chunk8.wav : In africa. 
audio-chunks/chunk9.wav : Say to myself you know like when i grow up and i can get rich. 
audio-chunks/chunk10.wav : Kids all over the world. 
audio-chunks/chunk11.wav : I will be in the position to do that by the time i was a teenager. 
Error: 
audio-chunks/chunk13.wav : 17 i started my career here in america. 
audio-chunks/chunk14.wav : Age of 18 i started my first charity. 
Error: 
audio-chunks/chunk16.wav : Team up with other organizations in the following years and met helped. 
Error: 
audio-chunks/chunk18.wav : Beautiful souls. 
audio-chunks/chunk19.wav : Girl yasmina amina. 
audi

## Summery the text from wav audio file using distilbart-cnn-12-6 model

In [16]:
import transformers
from transformers import pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

summery = summarizer(FullTextAudio)[0]['summary_text']
print(summery)

 i would see these commercials and i was watching other children suffer in other parts of the world. I would be in the position to do that by the time i was a teenager . Age of 18 i started my first charity. Team up with other organizations in the following years and met helped .


## Make a video from the summarized text

In [ ]:
#Generate the video Prompts 1
video_path = SDpipeline.walk(
    prompts=[summery+", hyper realistic, rendering by octane, unreal engine —ar 16:9  ",
            summery+", hyper realistic, rendering by octane, unreal engine —ar 16:9  ",
            summery+", hyper realistic, rendering by octane, unreal engine —ar 16:9  ",
            summery+", hyper realistic, rendering by octane, unreal engine —ar 16:9  ",],
    seeds=[2001,3001,4001,50001],
    num_interpolation_steps=50,
    #height=1280,  # use multiples of 64 if > 512. Multiples of 8 if < 512.
    #width=720,   # use multiples of 64 if > 512. Multiples of 8 if < 512.
    output_dir='videosFromAudios',        # Where images/videos will be saved
    name='rihannaSpeech',        # Subdirectory of output_dir where images/videos will be saved
    audio_filepath='/content/subtitles_trim_audio.wav',    # Use your own file
    guidance_scale=8.5,         # Higher adheres to prompt more, lower lets model take the wheel
    num_inference_steps=50,     # Number of diffusion steps per image generated. 50 is good default
    fps=6
)

/usr/local/lib/python3.9/dist-packages/diffusers/models/unet_2d_condition.py:452: FutureWarning: Accessing `in_channels` directly via unet.in_channels is deprecated. Please use `unet.config.in_channels` instead
  if hasattr(upsample_block, "has_cross_attention") and upsample_block.has_cross_attention:


  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

...

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]